<a href="https://colab.research.google.com/github/Ashupathak2001/Prasunet_Task/blob/main/Stock_price_prediction_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# importing the libraries
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('SUZLON.NS',
                      start=start_date,
                      end=end_date,
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close",
             "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3368,2024-08-16,77.949997,80.500000,74.120003,79.949997,79.949997,155613726
3369,2024-08-19,80.639999,82.449997,79.209999,80.709999,80.709999,96552950
3370,2024-08-20,81.309998,81.580002,78.500000,79.430000,79.430000,54449596
3371,2024-08-21,79.400002,80.150002,78.610001,78.849998,78.849998,38869576
3372,2024-08-22,79.230003,79.589996,77.300003,77.570000,77.570000,41519353


A candlestick chart gives a clear picture of the increase and decrease in stock prices, so let’s visualize a candlestick chart of the data before moving further:

In [6]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],open=data["Open"],high=data["High"],low=data["Low"],close=data["Close"])])
figure.update_layout(title = "Suzlon energy limited stock price analysis", xaxis_rangeslider_visible=False)
figure.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Now let’s have a look at the correlation of all the columns with the Close column as it is the target column:

In [7]:
# correlation
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    1.000000
High         0.999475
Low          0.999439
Open         0.998779
Volume       0.060396
Date        -0.205106
Name: Close, dtype: float64


**Training LSTM for Stock Price Prediction**

Now I will start with training an LSTM model for predicting stock prices. I will first split the data into training and test sets:

In [8]:
x = data[["Open",'High',"Low","Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1,1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)


Now prepare a neural network architecture for LSTM

In [12]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(xtrain.shape[1],1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 4, 128)              │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 25)                  │           1,625 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

Now here is how we can train our neural network model for stock price prediction:

In [13]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=30)

Epoch 1/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - loss: 27.6145
Epoch 2/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 1.8902
Epoch 3/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 43s 9ms/step - loss: 3.2970
Epoch 4/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - loss: 1.4652
Epoch 5/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - loss: 2.5658
Epoch 6/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 43s 9ms/step - loss: 1.0698
Epoch 7/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 42s 10ms/step - loss: 1.1460
Epoch 8/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 1.5130
Epoch 9/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - loss: 1.0492
Epoch 10/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step - loss: 0.9150
Epoch 11/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 42s 8ms/step - loss: 0.8755
Epoch 12/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - loss: 0.9500
Epoch 13/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.6299
Epoch 14/30
2698/2698 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - loss: 1.5911
Epoch 15/30
2

In [22]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[82.08, 87.41, 85.87, 87347002]])
model.predict(features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


array([[82.933426]], dtype=float32)